In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import wikipedia
import wikipediaapi

from scrape_to_mongodb import (fetch_article, collect_articles)

In [ ]:
drink_list = wikipedia.search("drink", results=1000)
food_list = wikipedia.search("food", results=1000)
ancient_list = wikipedia.search("ancient", results=1000)
transportation_list = wikipedia.search("transportation", results=1000)
shows_list = wikipedia.search("shows", results=1000)
music_list = wikipedia.search("music", results=1000)
actors_list = wikipedia.search("actors", results=1000)
movies_list = wikipedia.search("movies", results=1000)
finance_list = wikipedia.search("finance", results=1000)
#
language_list = wikipedia.search("language", results=1000)
nature_list = wikipedia.search("nature", results=1000)
environment_list = wikipedia.search("environment", results=1000)
history_list = wikipedia.search("history", results=1000)
econ_list = wikipedia.search("economics", results=1000)
nature_list = wikipedia.search("nature", results=1000)
lit_list = wikipedia.search("literature", results=1000)
psych_list = wikipedia.search("psychology", results=1000)
behavioral = wikipedia.search("behavioral", results=1000)
global_list = wikipedia.search("global", results=1000)
cities_list = wikipedia.search("cities", results=1000)
anthropology = wikipedia.search("anthropology", results=1000)
#
art_culture = wikipedia.search("art culture", results=1000)
geo_places = wikipedia.search("geography places", results=1000)
health_fit = wikipedia.search("health fitness", results=1000)
history_events = wikipedia.search("history events", results=1000)
math = wikipedia.search("mathematics abstractions", results=1000)
natural_science = wikipedia.search("natural sciences nature", results=1000)
people_self = wikipedia.search("people self", results=1000)
philosophy = wikipedia.search("philosophy thinking", results=1000)
religion = wikipedia.search("religion spirituality", results=1000)
social_society = wikipedia.search("social sciences society", results=1000)
tech = wikipedia.search("technology applied sciences", results=1000)

In [ ]:
all_articles = (art_culture + geo_places + health_fit + history_events + 
                math + natural_science + people_self + philosophy + religion
                + social_society + tech + anthropology + cities_list + global_list
                + behavioral + psych_list + lit_list + nature_list + econ_list 
                + history_list + environment_list + nature_list + language_list
                + finance_list + movies_list + actors_list + music_list + shows_list
                + transportation_list + ancient_list + food_list + drink_list)

In [ ]:
all_articles_unique = list(set(all_articles))

In [ ]:
# import pickle
# with open('data/unique_articles.pkl', 'wb') as f:
#     pickle.dump(all_articles_unique, f)

In [ ]:
# ! python scrape_to_mongodb.py

## Clean_df creation

In [1]:
import pickle
import os
import requests
from flask import Flask, request, render_template, jsonify

import numpy as np
import pandas as pd
import textstat
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine, cdist

import gensim
import operator
from operator import itemgetter

In [2]:
import pymongo
from pymongo import MongoClient

mc = pymongo.MongoClient()
db = mc['wikicache']
articles = db['articles']

In [3]:
data = list(articles.find())
clean_df = pd.DataFrame(data)


In [4]:
clean_df.head()

,_id,summary,text,title,url
0,5d1fc2f4258b4b335c5b2093,Avans University of Applied Sciences (Dutch: A...,Avans University of Applied Sciences (Dutch: A...,Avans University of Applied Sciences,https://en.wikipedia.org/wiki/Avans_University...
1,5d1fc2f5258b4b335c5b2094,"Guy de Lussigny (30 August 1929 in Cambrai, no...","Guy de Lussigny (30 August 1929 in Cambrai, no...",Guy de Lussigny,https://en.wikipedia.org/wiki/Guy_de_Lussigny
2,5d1fc2f6258b4b335c5b2095,"In literature, a trope is a common plot conven...","In literature, a trope is a common plot conven...",Trope (literature),https://en.wikipedia.org/wiki/Trope_(literature)
3,5d1fc2f6258b4b335c5b2096,The Global Television Network (more commonly c...,The Global Television Network (more commonly c...,Global Television Network,https://en.wikipedia.org/wiki/Global_Televisio...
4,5d1fc2f7258b4b335c5b2097,Nickelodeon Guts (stylized as Nickelodeon GUTS...,Nickelodeon Guts (stylized as Nickelodeon GUTS...,Nickelodeon Guts,https://en.wikipedia.org/wiki/Nickelodeon_Guts


In [13]:
clean_df['title']

0                     Avans University of Applied Sciences
1                                          Guy de Lussigny
2                                       Trope (literature)
3                                Global Television Network
4                                         Nickelodeon Guts
5                                Theatre of ancient Greece
6                                       Bliss point (food)
7                      Transportation theory (mathematics)
8        Department of Planning and Environment (New So...
9               Capital University of Science & Technology
10                                       Itasca State Park
11       University of International Business and Econo...
12                                         Global Tel Link
13                                          Caucasian race
14                                          Actors Theater
15                               Indian Actors Association
16                                             Star Movi

In [5]:
len(clean_df['text'])

11978

In [33]:
clean_text = clean_df['text'].tolist()

In [38]:
len(clean_text)

11978

In [40]:
complexity = [textstat.flesch_kincaid_grade(doc) for doc in clean_text]

In [42]:
clean_df['score'] = pd.Series(complexity)

In [43]:
clean_df.head()

,_id,summary,text,title,url,score
0,5d1fc2f4258b4b335c5b2093,Avans University of Applied Sciences (Dutch: A...,Avans University of Applied Sciences (Dutch: A...,Avans University of Applied Sciences,https://en.wikipedia.org/wiki/Avans_University...,18.0
1,5d1fc2f5258b4b335c5b2094,"Guy de Lussigny (30 August 1929 in Cambrai, no...","Guy de Lussigny (30 August 1929 in Cambrai, no...",Guy de Lussigny,https://en.wikipedia.org/wiki/Guy_de_Lussigny,25.1
2,5d1fc2f6258b4b335c5b2095,"In literature, a trope is a common plot conven...","In literature, a trope is a common plot conven...",Trope (literature),https://en.wikipedia.org/wiki/Trope_(literature),16.6
3,5d1fc2f6258b4b335c5b2096,The Global Television Network (more commonly c...,The Global Television Network (more commonly c...,Global Television Network,https://en.wikipedia.org/wiki/Global_Televisio...,15.0
4,5d1fc2f7258b4b335c5b2097,Nickelodeon Guts (stylized as Nickelodeon GUTS...,Nickelodeon Guts (stylized as Nickelodeon GUTS...,Nickelodeon Guts,https://en.wikipedia.org/wiki/Nickelodeon_Guts,14.4


In [44]:
# with open('data/clean_df.pkl', 'wb') as f:
#     pickle.dump(clean_df, f)